In [2]:
from datetime import datetime #to get the current date
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import pandas as pd
import time
import numpy as np

In [17]:
pwd

'/Users/yangzhang/document/ds_projs/ppj_JobScraping'

In [61]:
def get_jobs(keyword, num_jobs, verbose, slp_time):
    options = webdriver.ChromeOptions()
    #options.add_argument('headless')
    driver = webdriver.Chrome('/Users/yangzhang/document/ds_projs/ppj_JobScraping/chromedriver', options=options)
    driver.set_window_size(1120, 1600)
    
    url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+"&locT=&locId&jobType="
    driver.get(url)
    jobs = []
    
    while len(jobs) < num_jobs:
        time.sleep(slp_time)
        
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass
        
        try:
            driver.find_element_by_css_selector('[alt="Close"]').click()
        except NoSuchElementException:
            pass
        
        job_buttons = driver.find_elements_by_class_name("jl")
        print(job_buttons)
        
        for job_button in job_buttons:
            
            print("Progress: {}".format('' + str(len(jobs)) + '/' + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break
                
            job_button.click()
            time.sleep(10)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    company = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)
                    
            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray salary"]').text
            except NoSuchElementException:
                salary_estimate = -1
                
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1
                
            if verbose:
                print('Job Title: {}'.format(title))
                print('Salary Estimate: {}'.format(salary_estimate))
                print("Rating: {}".format(rating))
                print('Company Name: {}'.format(company))
                print("Location: {}".format(location))
            
            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            #try:
            #    driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()
             #   try:
             #       headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
             #   except NoSuchElementException:
             #       headquarters = -1
             #   try:
             #       sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
             #   except NoSuchElementException:
            #        sector = -1
          #  except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
            #    headquarters = -1
            #    sector = -1
                
          #  if verbose:
           #     print("Headquarters: {}".format(headquarters))
          #      print("Sector: {}".format(sector))
          #      print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({
                "JobTitle": title,
                "Company": company,
                "Location": location,
                "SalaryEstimate": salary_estimate,
                "Rating": rating,
                "JobDescription": description
                #"Sector": sector,
                #"Headquarters": headquarters
            })
            
            #Clicking on the "next page" button
            try:
                driver.find_element_by_xpath('.//li[@class="next"]//a').click()
            except NoSuchElementException:
                print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
                break
        
        return pd.DataFrame(jobs)

In [62]:
get_jobs('data scientist', 13, verbose=True, slp_time=10)

 x out worked
[<selenium.webdriver.remote.webelement.WebElement (session="ffc27ebdbb31b814e13eff74cec2b8f7", element="e0533f0c-f53f-4c1b-861c-8f1494829915")>, <selenium.webdriver.remote.webelement.WebElement (session="ffc27ebdbb31b814e13eff74cec2b8f7", element="d3d84844-3d1c-4085-9d21-a1938b64d6ac")>, <selenium.webdriver.remote.webelement.WebElement (session="ffc27ebdbb31b814e13eff74cec2b8f7", element="a4f52f9d-d3f8-4ce9-9e0a-9496a7b061df")>, <selenium.webdriver.remote.webelement.WebElement (session="ffc27ebdbb31b814e13eff74cec2b8f7", element="9adf23a9-a650-485d-bf86-cdfc03af521d")>, <selenium.webdriver.remote.webelement.WebElement (session="ffc27ebdbb31b814e13eff74cec2b8f7", element="e7e1bf56-6eac-4f69-b4c7-f2e3723c7115")>, <selenium.webdriver.remote.webelement.WebElement (session="ffc27ebdbb31b814e13eff74cec2b8f7", element="c32090b3-4438-4b7d-a9ac-e86e3d29524c")>, <selenium.webdriver.remote.webelement.WebElement (session="ffc27ebdbb31b814e13eff74cec2b8f7", element="aaceaae5-f65c-4a19

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=86.0.4240.111)
